In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
BASE_DIR = 'data/100-bird-species/'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
VALIDATION_DIR = os.path.join(BASE_DIR, 'valid')

CATEGORIES = [i for i in os.listdir(TRAIN_DIR)] # 175

In [ ]:
for category in CATEGORIES:
    path =  os.path.join(TRAIN_DIR, category)
    for image in os.listdir(path):
        img = cv2.imread(os.path.join(path, image))
        print(img.shape)
        plt.imshow(img)
        break
    break

In [ ]:
train_generator = ImageDataGenerator(
    rescale=1. / 255,
).flow_from_directory(
    TRAIN_DIR,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical',
)

validation_generator = ImageDataGenerator(
    rescale = 1. / 255,
).flow_from_directory(
    TEST_DIR,
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical',
)

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(175, activation='softmax'))

model.compile(
    optimizer = optimizers.RMSprop(lr=0.001),
    loss = losses.categorical_crossentropy,
    metrics = ['accuracy'],
)

model.summary()

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = 739,
    validation_data = validation_generator,
    validation_steps = 27,
    epochs = 30,
)

In [ ]:
model.save('bird_species.h5')

In [ ]:
model.load_weights('bird_species.h5')

In [ ]:
for img in validation_generator:
    image = img[0]
    print(image.shape)
    break
prediction = model.predict(image)

In [ ]:
index = 30
plt.imshow(image[index])
print(CATEGORIES[np.argmax(prediction[index])])